# Классификация

Необходимо выбрать представление данных и набор признаков(обосновать представление и выбранные признаки на основе разведочного анализа) и провести каждый эксперимент не менее чем на трех различных моделях(прописать обоснование выбора именного таких моделей). 

Так же необходимо выбрать метрики оценки решения и обосновать почему вы считаете что такого набора будет достаточно и он покажет качество, стабильность модели, а также позволит продемонстрировать преимущества и недостатки используемых алгоритмом. 

Эксперимент должен состоять из 
- построения датасета
- разделения его на две части: тестовую и тренировочную
- проведения обучения моделей и их оценки

После этого необходимо описать полученные результаты, выводы и рекомендации к ним. 
Реализовать метод подбора оптимального набора гиперпараметров для моделей и аргументировать выбор конкретного алгоритма подбора оптимального набора гиперпараметров в сравнении с аналогами.

Для сдачи необходимы:
- код экспериментов
- обоснования
- выводы

In [57]:
import pandas as pd

In [144]:
filename = "pikabu_dataset_good.csv"
filename_2000 = "pikabu_clean_dataset_2000.csv"
filename_hot = "hot_dataset_processed.csv"

In [145]:
df = pd.read_csv(filename_2000)

In [146]:
df.columns

Index(['Title', 'Link', 'Date', 'Views', 'Author', 'Tags', 'AmountComments',
       'Rating', 'Text'],
      dtype='object')

Классификация:
    - бинарная
    - мультиклассовая
    
Бинарная:
    - популярная статья или нет по содержанию
    - популярная статья или нет по содержанию и дате
    
    Популярность = рейтинг? количество просмотров + комментарии? 
    
Мультиклассовая: 
(по теме, тема = тег?)
    - предсказание тегов по названию и статье
    - предсказание тегов по статье
    
NaiveBayes
WordToVec
Bert

Кластеризация:
    - по тегам на основе текстов  
    - по популярности на основе текстов


по результатам EDA:

среди самых популярных тегов беполезные:
- мой, текст, длиннопост, nan

среди самых популярных слов в текстах бесполезные:
- это, который, один, свой, весь, мой, самый, мочь, такой, тот, другой, наш, два

половина слов встречаются 1 или 2 раза

In [147]:
df_classification = df[['Text', 'Tags']].dropna()

In [148]:
X, y = df_classification["Text"], df_classification["Tags"]

In [149]:
from sklearn.feature_extraction.text import CountVectorizer

#vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7)
vectorizer = CountVectorizer(max_features=1000, min_df=5, max_df=0.7)
X_countVectorizer = vectorizer.fit_transform(X).toarray()

#vectorizer = CountVectorizer(max_features=150, min_df=5, max_df=0.7)
vectorizer = CountVectorizer(max_features=50, min_df=5, max_df=0.7)
y_countVectorizer = vectorizer.fit_transform(y).toarray()

In [150]:
print(X_countVectorizer.shape)
print(y_countVectorizer.shape)

(22071, 1000)
(22071, 50)


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X_tfIdf = tfidfconverter.fit_transform(X_countVectorizer).toarray()
#y_tfIdf = tfidfconverter.fit_transform(y_countVectorizer).toarray()

In [151]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_countVectorizer, y_countVectorizer, test_size=0.25, random_state=0)

In [153]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=500, random_state=0)
classifier.fit(X_train, y_train) 

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0, multi_class="ovo").fit(X_train, y_train)
y_pred = clf.predict(X_test)
clf.score(X_train, y_train)

In [154]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = classifier.predict(X_test)
#print(confusion_matrix(y_test,y_pred))
#print(classification_report(y_test,y_pred))
#print(accuracy_score(y_test, y_pred))

In [159]:
print(y_pred[19])
print(y_test[19])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0]


In [158]:
import numpy as np
np.sum(np.not_equal(y_test, y_pred))/float(y_test.size)

0.03007611453425154

In [157]:
import csv
import os
import scipy as sp

def llfun(act, pred):
    epsilon = 1e-15
    pred = sp.maximum(epsilon, pred)
    pred = sp.minimum(1-epsilon, pred)
    ll = sum(act*sp.log(pred) + sp.subtract(1,act)*sp.log(sp.subtract(1,pred)))
    ll = ll * -1.0/len(act)
    return ll

def main():
    scores = []
    for index in range(0, len(y_pred)):
        result = llfun(y_test[index], y_pred[index])
        scores.append(result)

    print(sum(scores) / len(scores)) # 0.0985725708595

if __name__ == '__main__':
    main()

c:\users\138904\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: scipy.maximum is deprecated and will be removed in SciPy 2.0.0, use numpy.maximum instead
  import sys
c:\users\138904\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: scipy.minimum is deprecated and will be removed in SciPy 2.0.0, use numpy.minimum instead
  
c:\users\138904\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: scipy.log is deprecated and will be removed in SciPy 2.0.0, use numpy.lib.scimath.log instead
  if __name__ == '__main__':
c:\users\138904\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: scipy.subtract is deprecated and will be removed in SciPy 2.0.0, use numpy.subtract instead
  if __name__ == '__main__':


1.0276545780382087


In [83]:
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer

clf = OneVsRestClassifier(XGBClassifier(n_jobs=-1, max_depth=4))

# You may need to use MultiLabelBinarizer to encode your variables from arrays [[x, y, z]] to a multilabel 
# format before training.
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(y_train)

clf.fit(X_train, y)

c:\users\138904\appdata\local\programs\python\python37\lib\site-packages\sklearn\multiclass.py:75: UserWarning: Label 0 is present in all training examples.
  str(classes[c]))


OneVsRestClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, gamma=None,
                                            gpu_id=None, importance_type='gain',
                                            interaction_constraints=None,
                                            learning_rate=None,
                                            max_delta_step=None, max_depth=4,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            n_estimators=100, n_jobs=-1,
                                            num_parallel_tree=None,
                                            objective='binary:logistic',
                                            rando

In [84]:
y_pred = clf.predict(X_test)

In [111]:
df[['Views']].head()

,Views
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [112]:
df[['Rating']]

,Rating
0,8.0
1,11.0
2,-13.0
3,9.0
4,-145.0
...,...
58249,-5.0
58250,582.0
58251,144.0
58252,NaN


In [136]:
import numpy as np
a = np.array(df[['Rating']].dropna().iloc[:,:])
p = np.percentile(a, 75) # return 50th percentile, e.g median.
print(p)

92.0


In [ ]:
from sklearn.metrics import hamming_loss
hamming_loss(y_test, y_pred)

In [ ]:
from sklearn.metrics import zero_one_loss
zero_one_loss(y_test, y_pred, normalize=False)

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred, multi_class='ovr')

In [ ]:
from sklearn import preprocessing

def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = preprocessing.LabelBinarizer()
    lb.fit(y_test)
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    return roc_auc_score(y_test, y_pred, average=average)

multiclass_roc_auc_score(y_test, y_pred)